In [1]:
import numpy as np
import pandas as pd

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-fusion-2024-churn-features-6-3/main_ft_065.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/main_ft_060.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/__results__.html
/kaggle/input/data-fusion-2024-churn-features-6-3/main_tsf_055.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/main_tsf_065.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/main_tsf.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/main_ft_055.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/main_ft.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/main_tsf_060.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/__notebook__.ipynb
/kaggle/input/data-fusion-2024-churn-features-6-3/main_ft_050.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/__output__.json
/kaggle/input/data-fusion-2024-churn-features-6-3/main_tsf_050.csv
/kaggle/input/data-fusion-2024-churn-features-6-3/custom.css
/kaggle/input/data-fusion-2024-churn-features-6-3/catboost_info/learn_error.tsv


In [2]:
import pandas as pd
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from IPython.display import display, HTML

pd.set_option('display.max_columns', None)
warnings.simplefilter('ignore')
# pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
random.seed(42)
np.random.seed(42)

#### Predict Catboost добавление доп фичей последовательностей 6.2 - 6.6 ПЛЮС фичи Марка ПЛЮС доп генерация фичей методами Марка И ЕЩЕ ПЛЮС фичи из базового ноута, но на модификациях последовательностей транзакций

In [3]:
import numpy as np
import pandas as pd

import os



import pandas as pd
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from IPython.display import display, HTML

pd.set_option('display.max_columns', None)
warnings.simplefilter('ignore')
# pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
random.seed(42)
np.random.seed(42)

In [4]:
# Фичи Марка

In [5]:
from tqdm import  tqdm

# 1

clients = pd.read_parquet('/kaggle/input/marks-features/clients.parquet') # фичи Марка
# clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
clients_features_all = pd.read_parquet('/kaggle/input/marks-features/clients_features_all.parquet') # наши фичи
tran_train = pd.read_parquet('/kaggle/input/marks-features/tran_train.parquet') # транзакции с доп столбцами
transactions = pd.read_parquet('/kaggle/input/marks-features/transactions.parquet') # транзакции с доп столбцами


In [6]:
base_clients_columns = clients.columns.to_list()

In [7]:

def current_rk_col(clients, transactions, method_list=['median'], curr_list=[1, 0, 2, 3]):
    for ex in tqdm(curr_list):
        for method in method_list:
            average_per_user = transactions.loc[transactions.currency_rk==ex].groupby('user_id').transaction_amt.agg(method)
            clients[f'rk{ex}{method}'] = 0 
            clients.loc[clients.user_id.isin(average_per_user.index), f'rk{ex}{method}'] = average_per_user.values
            
    return clients, transactions

def mcc_code_col(clients, transactions, method_list=['mean'], mcc_list=[9, 3, 155, 12, 1, 0, 26]):
    for ex in tqdm(mcc_list):
        for method in method_list:
            average_per_user = transactions.loc[transactions.mcc_code==ex].groupby('user_id').transaction_amt.agg(method)
            clients[f'mcc_code{ex}{method}'] = 0 
            clients.loc[clients.user_id.isin(average_per_user.index), f'mcc_code{ex}{method}'] = average_per_user.values
            
    return clients, transactions

def mcc_code_cr_col(clients, transactions, method_list=['std'], mcc_list=[9, 3, 155, 12, 1, 0, 26]):
    for ex in tqdm([1, 0]):
        for method in method_list:
            for mcc_code in mcc_list:
                search = (transactions.currency_rk==ex)&(transactions.mcc_code==mcc_code)
                average_per_user = transactions.loc[search].groupby('user_id').transaction_amt.agg(method)
                clients[f'rk{ex}{method}{mcc_code}'] = 0 
                clients.loc[clients.user_id.isin(average_per_user.index), f'rk{ex}{method}{mcc_code}'] = average_per_user.values
    return clients, transactions
     
def week_data_cr_col(clients, transactions, method_list=['std']):
    for ex in tqdm([1, 0]):
        for ex1 in range(transactions.week_number.unique().size):
            for method in method_list:
                search = (transactions.week_number==ex1+1)&(transactions.currency_rk==ex)
                average_per_user = transactions.loc[search].groupby('user_id').transaction_amt.agg(method)
                clients[f'month{ex1+1}_{method}{ex}'] = 0 
                clients.loc[clients.user_id.isin(average_per_user.index), f'month{ex1+1}_{method}{ex}'] = average_per_user.values
                
    return clients, transactions

train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
clients = clients.drop(columns=['mega_features1',"mega_features",
                                'group_employee_age_mean', 'Unnamed: 0'])

method_list = ['mean', 'sum', 'std']
clients, transactions = current_rk_col(clients, transactions, method_list=method_list, curr_list=[1, 0, 2, 3])
clients, transactions = mcc_code_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = mcc_code_cr_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = week_data_cr_col(clients, transactions, method_list=method_list)

100%|██████████| 2/2 [00:07<00:00,  3.88s/it]


In [8]:
new_cl_cols = [el for el in clients.columns if el not in base_clients_columns]

In [9]:
print(clients.shape)

(96000, 197)


In [10]:
clients['customer_age_procces'] = clients['customer_age'].replace({1:2, 3:2})
clients['employee_count_nm_process'] = clients['employee_count_nm'].replace({5:6, 1:6})
clients['mega_features'] = clients['customer_age_procces'].astype(str)  + clients['employee_count_nm_process'].astype(str)

for v in tqdm(clients['mega_features'].unique()):
    df1 = clients.loc[(clients.mega_features==v)&(clients.target!=-1)]
    clients.loc[clients.mega_features==v, 'mega_features'] = df1['target'].mean()
   
for ex in range(transactions.week_number.unique().size):
    for method in ['mean',]:
        average_per_user = transactions.loc[(transactions.week_number==ex+1)&(transactions.currency_rk==1)].groupby('user_id').transaction_amt.agg(method)
        clients[f'month{ex+1}1_{method}'] = 0 
        new_cl_cols.append(f'month{ex+1}1_{method}')
        clients.loc[clients.user_id.isin(average_per_user.index), f'month{ex+1}1_{method}'] = average_per_user.values
        
clients['mega_features'] = clients['mega_features'].astype(float)

100%|██████████| 16/16 [00:09<00:00,  1.70it/s]


In [11]:
print(clients.shape)

(96000, 214)


In [12]:
clients = clients.drop(columns=['target', 'user_id_customer_age',
                                'employee_count_nm_process', 'customer_age_procces'])

In [13]:
len(set(clients.columns))

210

In [14]:
for col in new_cl_cols:
    clients.rename(columns={col: 'seq_1_'+col}, inplace=True)

In [15]:
clients_1 = clients.copy()

In [16]:
# 2

clients = pd.read_parquet('/kaggle/input/marks-features/clients.parquet') # фичи Марка
# clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
clients_features_all = pd.read_parquet('/kaggle/input/marks-features/clients_features_all.parquet') # наши фичи
tran_train = pd.read_parquet('/kaggle/input/marks-features/tran_train.parquet') # транзакции с доп столбцами
transactions = pd.read_parquet('/kaggle/input/marks-features/transactions.parquet') # транзакции с доп столбцами

clients_true = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])

transactions = transactions.merge(clients_true[['user_id', 'report']], how='left').merge(report_dates, how='left')
transactions['report_delta'] = (transactions['report_dt'] - transactions['transaction_dttm']).dt.total_seconds().fillna(0) / (3600*24)
transactions['transaction_amt'] = transactions['report_delta']
transactions = transactions.drop(['report', 'report_dt', 'report_delta'], axis=1)


base_clients_columns = clients.columns.to_list()


train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
clients = clients.drop(columns=['mega_features1',"mega_features",
                                'group_employee_age_mean', 'Unnamed: 0'])

method_list = ['mean', 'sum', 'std']
clients, transactions = current_rk_col(clients, transactions, method_list=method_list, curr_list=[1, 0, 2, 3])
clients, transactions = mcc_code_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = mcc_code_cr_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = week_data_cr_col(clients, transactions, method_list=method_list)

new_cl_cols = [el for el in clients.columns if el not in base_clients_columns]

clients['customer_age_procces'] = clients['customer_age'].replace({1:2, 3:2})
clients['employee_count_nm_process'] = clients['employee_count_nm'].replace({5:6, 1:6})
clients['mega_features'] = clients['customer_age_procces'].astype(str)  + clients['employee_count_nm_process'].astype(str)

for v in tqdm(clients['mega_features'].unique()):
    df1 = clients.loc[(clients.mega_features==v)&(clients.target!=-1)]
    clients.loc[clients.mega_features==v, 'mega_features'] = df1['target'].mean()
   
for ex in range(transactions.week_number.unique().size):
    for method in ['mean',]:
        average_per_user = transactions.loc[(transactions.week_number==ex+1)&(transactions.currency_rk==1)].groupby('user_id').transaction_amt.agg(method)
        clients[f'month{ex+1}1_{method}'] = 0 
        new_cl_cols.append(f'month{ex+1}1_{method}')
        clients.loc[clients.user_id.isin(average_per_user.index), f'month{ex+1}1_{method}'] = average_per_user.values
        
clients['mega_features'] = clients['mega_features'].astype(float)

clients = clients.drop(columns=['target', 'user_id_customer_age',
                                'employee_count_nm_process', 'customer_age_procces'])

new_cl_cols_renamed = []
for col in new_cl_cols:
    clients.rename(columns={col: 'seq_2_'+col}, inplace=True)
    new_cl_cols_renamed.append('seq_2_'+col)
    
clients = clients[['user_id']+new_cl_cols_renamed]
    
clients_2 = clients.copy()

100%|██████████| 16/16 [00:08<00:00,  1.86it/s]


In [17]:
# 3

clients = pd.read_parquet('/kaggle/input/marks-features/clients.parquet') # фичи Марка
# clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
clients_features_all = pd.read_parquet('/kaggle/input/marks-features/clients_features_all.parquet') # наши фичи
tran_train = pd.read_parquet('/kaggle/input/marks-features/tran_train.parquet') # транзакции с доп столбцами
transactions = pd.read_parquet('/kaggle/input/marks-features/transactions.parquet') # транзакции с доп столбцами

clients_true = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])

transactions['min_trans_dt'] = transactions.groupby(['user_id'])['transaction_dttm'].transform('min')
transactions['delta'] = (transactions['transaction_dttm'] - transactions['min_trans_dt']).dt.total_seconds().fillna(0) / (3600*24)
transactions['transaction_amt'] = transactions['delta']
transactions = transactions.drop(['min_trans_dt', 'delta'], axis=1)


base_clients_columns = clients.columns.to_list()


train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
clients = clients.drop(columns=['mega_features1',"mega_features",
                                'group_employee_age_mean', 'Unnamed: 0'])

method_list = ['mean', 'sum', 'std']
clients, transactions = current_rk_col(clients, transactions, method_list=method_list, curr_list=[1, 0, 2, 3])
clients, transactions = mcc_code_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = mcc_code_cr_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = week_data_cr_col(clients, transactions, method_list=method_list)

new_cl_cols = [el for el in clients.columns if el not in base_clients_columns]

clients['customer_age_procces'] = clients['customer_age'].replace({1:2, 3:2})
clients['employee_count_nm_process'] = clients['employee_count_nm'].replace({5:6, 1:6})
clients['mega_features'] = clients['customer_age_procces'].astype(str)  + clients['employee_count_nm_process'].astype(str)

for v in tqdm(clients['mega_features'].unique()):
    df1 = clients.loc[(clients.mega_features==v)&(clients.target!=-1)]
    clients.loc[clients.mega_features==v, 'mega_features'] = df1['target'].mean()
   
for ex in range(transactions.week_number.unique().size):
    for method in ['mean',]:
        average_per_user = transactions.loc[(transactions.week_number==ex+1)&(transactions.currency_rk==1)].groupby('user_id').transaction_amt.agg(method)
        clients[f'month{ex+1}1_{method}'] = 0 
        new_cl_cols.append(f'month{ex+1}1_{method}')
        clients.loc[clients.user_id.isin(average_per_user.index), f'month{ex+1}1_{method}'] = average_per_user.values
        
clients['mega_features'] = clients['mega_features'].astype(float)

clients = clients.drop(columns=['target', 'user_id_customer_age',
                                'employee_count_nm_process', 'customer_age_procces'])

new_cl_cols_renamed = []
for col in new_cl_cols:
    clients.rename(columns={col: 'seq_3_'+col}, inplace=True)
    new_cl_cols_renamed.append('seq_3_'+col)
    
clients = clients[['user_id']+new_cl_cols_renamed]
    
clients_3 = clients.copy()

100%|██████████| 16/16 [00:08<00:00,  1.84it/s]


In [18]:
# 4

clients = pd.read_parquet('/kaggle/input/marks-features/clients.parquet') # фичи Марка
# clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
clients_features_all = pd.read_parquet('/kaggle/input/marks-features/clients_features_all.parquet') # наши фичи
tran_train = pd.read_parquet('/kaggle/input/marks-features/tran_train.parquet') # транзакции с доп столбцами
transactions = pd.read_parquet('/kaggle/input/marks-features/transactions.parquet') # транзакции с доп столбцами

clients_true = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])

transactions['transaction_amt'] = transactions['mcc_code']


base_clients_columns = clients.columns.to_list()


train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
clients = clients.drop(columns=['mega_features1',"mega_features",
                                'group_employee_age_mean', 'Unnamed: 0'])

method_list = ['mean', 'sum', 'std']
clients, transactions = current_rk_col(clients, transactions, method_list=method_list, curr_list=[1, 0, 2, 3])
clients, transactions = mcc_code_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = mcc_code_cr_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = week_data_cr_col(clients, transactions, method_list=method_list)

new_cl_cols = [el for el in clients.columns if el not in base_clients_columns]

clients['customer_age_procces'] = clients['customer_age'].replace({1:2, 3:2})
clients['employee_count_nm_process'] = clients['employee_count_nm'].replace({5:6, 1:6})
clients['mega_features'] = clients['customer_age_procces'].astype(str)  + clients['employee_count_nm_process'].astype(str)

for v in tqdm(clients['mega_features'].unique()):
    df1 = clients.loc[(clients.mega_features==v)&(clients.target!=-1)]
    clients.loc[clients.mega_features==v, 'mega_features'] = df1['target'].mean()
   
for ex in range(transactions.week_number.unique().size):
    for method in ['mean',]:
        average_per_user = transactions.loc[(transactions.week_number==ex+1)&(transactions.currency_rk==1)].groupby('user_id').transaction_amt.agg(method)
        clients[f'month{ex+1}1_{method}'] = 0 
        new_cl_cols.append(f'month{ex+1}1_{method}')
        clients.loc[clients.user_id.isin(average_per_user.index), f'month{ex+1}1_{method}'] = average_per_user.values
        
clients['mega_features'] = clients['mega_features'].astype(float)

clients = clients.drop(columns=['target', 'user_id_customer_age',
                                'employee_count_nm_process', 'customer_age_procces'])

new_cl_cols_renamed = []
for col in new_cl_cols:
    clients.rename(columns={col: 'seq_4_'+col}, inplace=True)
    new_cl_cols_renamed.append('seq_4_'+col)
    
clients = clients[['user_id']+new_cl_cols_renamed]
    
clients_4 = clients.copy()

100%|██████████| 16/16 [00:09<00:00,  1.76it/s]


In [19]:
# 5

clients = pd.read_parquet('/kaggle/input/marks-features/clients.parquet') # фичи Марка
# clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
clients_features_all = pd.read_parquet('/kaggle/input/marks-features/clients_features_all.parquet') # наши фичи
tran_train = pd.read_parquet('/kaggle/input/marks-features/tran_train.parquet') # транзакции с доп столбцами
transactions = pd.read_parquet('/kaggle/input/marks-features/transactions.parquet') # транзакции с доп столбцами

clients_true = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])

transactions['time_delta'] = transactions.groupby('user_id')['transaction_dttm'].diff().dt.total_seconds().fillna(0) / (3600*24)
transactions['transaction_amt'] = transactions['time_delta']
transactions = transactions.drop(['time_delta'], axis=1)


base_clients_columns = clients.columns.to_list()


train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
clients = clients.drop(columns=['mega_features1',"mega_features",
                                'group_employee_age_mean', 'Unnamed: 0'])

method_list = ['mean', 'sum', 'std']
clients, transactions = current_rk_col(clients, transactions, method_list=method_list, curr_list=[1, 0, 2, 3])
clients, transactions = mcc_code_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = mcc_code_cr_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = week_data_cr_col(clients, transactions, method_list=method_list)

new_cl_cols = [el for el in clients.columns if el not in base_clients_columns]

clients['customer_age_procces'] = clients['customer_age'].replace({1:2, 3:2})
clients['employee_count_nm_process'] = clients['employee_count_nm'].replace({5:6, 1:6})
clients['mega_features'] = clients['customer_age_procces'].astype(str)  + clients['employee_count_nm_process'].astype(str)

for v in tqdm(clients['mega_features'].unique()):
    df1 = clients.loc[(clients.mega_features==v)&(clients.target!=-1)]
    clients.loc[clients.mega_features==v, 'mega_features'] = df1['target'].mean()
   
for ex in range(transactions.week_number.unique().size):
    for method in ['mean',]:
        average_per_user = transactions.loc[(transactions.week_number==ex+1)&(transactions.currency_rk==1)].groupby('user_id').transaction_amt.agg(method)
        clients[f'month{ex+1}1_{method}'] = 0 
        new_cl_cols.append(f'month{ex+1}1_{method}')
        clients.loc[clients.user_id.isin(average_per_user.index), f'month{ex+1}1_{method}'] = average_per_user.values
        
clients['mega_features'] = clients['mega_features'].astype(float)

clients = clients.drop(columns=['target', 'user_id_customer_age',
                                'employee_count_nm_process', 'customer_age_procces'])

new_cl_cols_renamed = []
for col in new_cl_cols:
    clients.rename(columns={col: 'seq_5_'+col}, inplace=True)
    new_cl_cols_renamed.append('seq_5_'+col)
    
clients = clients[['user_id']+new_cl_cols_renamed]
    
clients_5 = clients.copy()

100%|██████████| 16/16 [00:09<00:00,  1.74it/s]


In [20]:
# 6

clients = pd.read_parquet('/kaggle/input/marks-features/clients.parquet') # фичи Марка
# clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
clients_features_all = pd.read_parquet('/kaggle/input/marks-features/clients_features_all.parquet') # наши фичи
tran_train = pd.read_parquet('/kaggle/input/marks-features/tran_train.parquet') # транзакции с доп столбцами
transactions = pd.read_parquet('/kaggle/input/marks-features/transactions.parquet') # транзакции с доп столбцами

clients_true = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
report_dates = pd.read_csv('/kaggle/input/ods-churn-24/report_dates.csv', parse_dates=['report_dt'])

transactions = transactions.sort_values(by=['user_id', 'transaction_dttm'])
transactions['cumsum'] = transactions.groupby('user_id')['transaction_amt'].cumsum()
transactions['transaction_amt'] = transactions['cumsum']
transactions = transactions.drop(['cumsum'], axis=1)


base_clients_columns = clients.columns.to_list()


train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
clients = clients.drop(columns=['mega_features1',"mega_features",
                                'group_employee_age_mean', 'Unnamed: 0'])

method_list = ['mean', 'sum', 'std']
clients, transactions = current_rk_col(clients, transactions, method_list=method_list, curr_list=[1, 0, 2, 3])
clients, transactions = mcc_code_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = mcc_code_cr_col(clients, transactions, method_list=method_list, mcc_list=[9, 3, 155, 12, 1, 0, 26])
clients, transactions = week_data_cr_col(clients, transactions, method_list=method_list)

new_cl_cols = [el for el in clients.columns if el not in base_clients_columns]

clients['customer_age_procces'] = clients['customer_age'].replace({1:2, 3:2})
clients['employee_count_nm_process'] = clients['employee_count_nm'].replace({5:6, 1:6})
clients['mega_features'] = clients['customer_age_procces'].astype(str)  + clients['employee_count_nm_process'].astype(str)

for v in tqdm(clients['mega_features'].unique()):
    df1 = clients.loc[(clients.mega_features==v)&(clients.target!=-1)]
    clients.loc[clients.mega_features==v, 'mega_features'] = df1['target'].mean()
   
for ex in range(transactions.week_number.unique().size):
    for method in ['mean',]:
        average_per_user = transactions.loc[(transactions.week_number==ex+1)&(transactions.currency_rk==1)].groupby('user_id').transaction_amt.agg(method)
        clients[f'month{ex+1}1_{method}'] = 0 
        new_cl_cols.append(f'month{ex+1}1_{method}')
        clients.loc[clients.user_id.isin(average_per_user.index), f'month{ex+1}1_{method}'] = average_per_user.values
        
clients['mega_features'] = clients['mega_features'].astype(float)

clients = clients.drop(columns=['target', 'user_id_customer_age',
                                'employee_count_nm_process', 'customer_age_procces'])

new_cl_cols_renamed = []
for col in new_cl_cols:
    clients.rename(columns={col: 'seq_6_'+col}, inplace=True)
    new_cl_cols_renamed.append('seq_6_'+col)
    
clients = clients[['user_id']+new_cl_cols_renamed]
    
clients_6 = clients.copy()

100%|██████████| 16/16 [00:08<00:00,  1.82it/s]


In [21]:
# clients = clients_1.copy()
# clients = (clients.merge(clients_2, how='left').merge(clients_3, how='left')
#                   .merge(clients_4, how='left').merge(clients_5, how='left')
#                   .merge(clients_6, how='left'))


clients = clients_2.copy()
clients = (clients.merge(clients_3, how='left')
                  .merge(clients_4, how='left').merge(clients_5, how='left')
                  .merge(clients_6, how='left'))

In [22]:
clients.shape

(96000, 866)

In [23]:
# Отсев фич Марка

In [24]:
clients.replace([np.inf, -np.inf], np.nan, inplace=True)
clients.fillna(0, inplace=True)

train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = clients.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)

m_cat_cols = []
for col in clients.columns:
    if col not in ['user_id', 'target', 'time']:
        if clients[col].nunique() < 15 and clients[col].nunique() > 1:
            m_cat_cols.append(col)
train[m_cat_cols] = train[m_cat_cols].astype('str')

# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    eval_metric='AUC',
    cat_features = m_cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)

df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.5] # 0.5

m_new_good_cols = df_imp['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))


clients = clients[['user_id']+m_new_good_cols]

clients = clients.merge(clients_1, how='left')

TBB Warning: The number of workers is currently limited to 3. The request for 5 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	total: 306ms	remaining: 7m 7s
100:	total: 24s	remaining: 5m 9s
200:	total: 47.2s	remaining: 4m 41s
300:	total: 1m 11s	remaining: 4m 22s
400:	total: 1m 32s	remaining: 3m 51s
500:	total: 1m 53s	remaining: 3m 23s
600:	total: 2m 19s	remaining: 3m 5s
700:	total: 2m 41s	remaining: 2m 40s
800:	total: 3m 2s	remaining: 2m 16s
900:	total: 3m 23s	remaining: 1m 52s
1000:	total: 3m 43s	remaining: 1m 29s
1100:	total: 4m 5s	remaining: 1m 6s
1200:	total: 4m 26s	remaining: 44.1s
1300:	total: 4m 46s	remaining: 21.8s
1399:	total: 5m 8s	remaining: 0us
0.8427337824460212


In [25]:
m_new_good_cols

['seq_6_rk1std',
 'seq_6_month12_std1',
 'seq_4_rk1std',
 'seq_5_rk1mean',
 'seq_2_rk1sum',
 'seq_2_rk1std',
 'seq_2_mcc_code0sum',
 'seq_3_rk1std',
 'seq_4_rk1mean',
 'seq_6_month11_std1',
 'seq_2_rk1mean',
 'seq_6_month12_mean1',
 'seq_6_rk1mean0',
 'seq_5_rk1sum',
 'seq_5_month8_std1',
 'seq_2_mcc_code12std',
 'seq_6_mcc_code0mean',
 'seq_2_rk1mean3',
 'seq_3_month81_mean',
 'seq_2_rk1sum0',
 'seq_6_mcc_code12std',
 'seq_6_month10_std1',
 'seq_6_mcc_code1std',
 'seq_6_month4_mean1']

In [26]:
# clients

In [27]:
# Фичи Андрея

In [28]:
main_621 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-2/main_ft.csv')
main_622 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-2/main_tsf.csv')
main_631 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-3/main_ft.csv')
main_632 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-3/main_tsf.csv')
main_641 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-4/main_ft.csv')
main_642 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-4/main_tsf.csv')
main_651 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-5/main_ft.csv')
main_652 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-5/main_tsf.csv')
main_661 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-6/main_ft.csv')
main_662 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-6/main_tsf.csv')

main_671 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-7/main_ft.csv')
main_681 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-8/main_ft.csv')
main_682 = pd.read_csv('/kaggle/input/data-fusion-2024-churn-features-6-8/main_tsf.csv')

main_62 = main_621.merge(main_622, how='left')
main_63 = main_631.merge(main_632, how='left')
main_64 = main_641.merge(main_642, how='left')
main_65 = main_651.merge(main_652, how='left')
main_66 = main_661.merge(main_662, how='left')
main_67 = main_671.copy()
main_68 = main_681.merge(main_682, how='left')
main_62.columns = ['user_id']+['main_62_'+str(i) for i in range(main_62.shape[1]-1)]
main_63.columns = ['user_id']+['main_63_'+str(i) for i in range(main_63.shape[1]-1)]
main_64.columns = ['user_id']+['main_64_'+str(i) for i in range(main_64.shape[1]-1)]
main_65.columns = ['user_id']+['main_65_'+str(i) for i in range(main_65.shape[1]-1)]
main_66.columns = ['user_id']+['main_66_'+str(i) for i in range(main_66.shape[1]-1)]
main_67.columns = ['user_id']+['main_67_'+str(i) for i in range(main_67.shape[1]-1)]
main_68.columns = ['user_id']+['main_68_'+str(i) for i in range(main_68.shape[1]-1)]

# main = main_62.merge(main_63, how='left').merge(main_64, how='left').merge(main_65, how='left').merge(main_66, how='left')

# new_good_cols = ['main_62_2', 'main_66_102', 'main_62_1', 'main_64_3', 'main_64_101',
#                  'main_66_99', 'main_62_8', 'main_66_1', 'main_62_11', 'main_62_5',
#                  'main_63_80', 'main_64_95', 'main_66_105', 'main_62_0', 'main_66_4',
#                  'main_65_84', 'main_64_2', 'main_63_2', 'main_64_99', 'main_62_84']

# main = main[['user_id']+new_good_cols]



main = main_62.merge(main_63, how='left').merge(main_64, how='left').merge(main_65, how='left').merge(main_66, how='left').merge(main_67, how='left').merge(main_68, how='left')

train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
train = main.merge(train[['user_id', 'target']], how='left')
train = train[~train.target.isna()]
train = train.sort_values('user_id').reset_index(drop=True)


# Обучение модельки для того чтобы получить важные фичи

model = CatBoostClassifier(
    iterations = 1400,
    depth=5,
    learning_rate=0.03,
    
    eval_metric='AUC',
#     cat_features = cat_cols,
    thread_count=6,
    early_stopping_rounds=200,
)
model.fit(train.drop(['user_id', 'target'], axis=1), train['target'], verbose=100)

df_imp = pd.DataFrame({
    'name': train.drop(['user_id', 'target'], axis=1).columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

df_imp = df_imp[df_imp['imp'] > 0.3]

new_good_cols = df_imp['name'].tolist()

pred = model.predict_proba(train.drop(['user_id', 'target'], axis=1))[:, 1]
print(metrics.roc_auc_score(train['target'], pred))



0:	total: 527ms	remaining: 12m 17s
100:	total: 55.4s	remaining: 11m 52s
200:	total: 1m 38s	remaining: 9m 46s
300:	total: 2m 19s	remaining: 8m 30s
400:	total: 3m 1s	remaining: 7m 32s
500:	total: 3m 40s	remaining: 6m 35s
600:	total: 4m 19s	remaining: 5m 45s
700:	total: 4m 58s	remaining: 4m 57s
800:	total: 5m 36s	remaining: 4m 11s
900:	total: 6m 15s	remaining: 3m 27s
1000:	total: 6m 54s	remaining: 2m 45s
1100:	total: 7m 36s	remaining: 2m 3s
1200:	total: 8m 15s	remaining: 1m 22s
1300:	total: 8m 54s	remaining: 40.7s
1399:	total: 9m 32s	remaining: 0us
0.8764817255424034


In [35]:
main[new_good_cols].mean().sort_values()

main = main[['user_id']+new_good_cols]

duplicates = ['main_62_83', 'main_63_1', 'main_62_2', 'main_65_0', 'main_64_1', 'main_63_0', 'main_66_0']
main = main[[el for el in main.columns if el not in duplicates]]
new_good_cols = [el for el in new_good_cols if el not in duplicates]

In [36]:
df = pd.read_parquet('/kaggle/input/data-fusion-2024-churn/df.parquet')
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
df = df.merge(train, how='left')
df.count_trans_date_max = df.count_trans_date_max.astype(int)

cat_cols = ['count_trans_date_max', 'employee_count_nm', 'employee_count_freq', 'PERCENTILE(NUM_UNIQUE(transactions.MONTH(transaction_dttm)))']
good_cols = [el for el in df.columns if el not in ['user_id', 'time', 'target']]

df = df.merge(main, how='left')
good_cols += new_good_cols

In [37]:
clients = clients[['user_id']+[el for el in clients.columns if el not in df.columns.to_list()+['report', 'bankemplstatus', 'customer_age']]]
m_cat_cols = []
for col in clients.columns:
    if col not in ['user_id', 'target', 'time']:
        if clients[col].nunique() < 15 and clients[col].nunique() > 1:
            m_cat_cols.append(col)
print(m_cat_cols)

['user_id1', 'currency_rk_nunique', 'mean', 'seq_1_month13_std0']


In [38]:
add_features_from_base = pd.read_parquet('/kaggle/input/data-fusion-2024-churn-additional-trans-features/df_add.parquet')
add_cols = [el for el in add_features_from_base.columns if el != 'user_id']

In [39]:
from collections import Counter
d = Counter(add_cols)
d

Counter({'^2_avg_trans_per_mcc_mult_^5_currency_max_1': 1,
         '^5_currency_max_1_mult_^2_avg_trans_per_mcc': 1,
         '^2_avg_trans_per_mcc_mult_^2_currency_min_1': 1,
         '^2_mcc3': 1,
         '^4_monthly_trans_mean': 1,
         '^4_currency_max_1_mult_^6_trans_amount_std': 1,
         '^6_trans_amount_std_mult_^6_currency_std_1': 1,
         '^6_currency_std_1_mult_^2_currency_daydiff_30_mean_1': 1,
         '^3_currency_std_1': 1,
         '^5_hour_median_2': 1,
         '^2_currency_daydiff_30_mean_1_mult_^6_currency_std_1': 1,
         '^2_currency_std_1': 1,
         '^6_currency_std_1_mult_^3_trans_amount_std': 1,
         '^2_currency_daydiff_30_mean_1_mult_^5_trans_amount_age_interaction3': 1,
         '^5_trans_amt_neg_sum_0_15': 1,
         '^6_currency_std_1_mult_^5_currency_max_1': 1,
         'count_trans_^6_ss^_^median': 1,
         '^5_mcc15': 1,
         '^5_trans_amount_age_interaction2': 1,
         '^2_currency_daydiff_30_mean_1': 1,
         '^5_tra

In [40]:
len(add_cols)

36

In [41]:
df = df.merge(clients, how='left', on='user_id')
df = df.merge(add_features_from_base, how='left', on='user_id')

good_cols += m_cat_cols
good_cols = [el for el in df.columns if el not in ['user_id', 'target', 'time', 'report', 'customer_age', 'bankemplstatus']]
cat_cols += m_cat_cols

# good_cols += add_cols


df.loc[df.target.isna(), 'target'] = -1

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

train = df[df.target != -1]
test = df[df.target == -1]

In [42]:
df.drop(['target', 'time'], axis=1).to_parquet('df_all.parquet', index=False)

In [43]:
len(good_cols)

440

In [44]:
train[good_cols].head(3)

,CUM_SUM(COUNT(transactions)),currency_std_1_mult_employee_count_nm,"CUM_SUM(TREND(transactions.transaction_amt, transaction_dttm))",CUM_SUM(MEAN(transactions.DIFF(transaction_amt))),CUM_SUM(MIN(transactions.TIME_SINCE_PREVIOUS(transaction_dttm))),employee_count_nm_mult_max_min_diff_dttm,employee_count_nm_mult_percent_last_10,"transaction_amt__fft_aggregated__aggtype_""kurtosis""","date_('interval', 'max')",msumm4,days_since_last_trans,main_count_transaction_amt_155_norm,count_trans_date_max,main_count_transaction_amt_3_norm,max_min_diff_dttm_mult_employee_count_nm,report_mult_max_min_diff_dttm,trans_amt_sum_0_15_ratio,max_min_diff_dttm_mult_report,percent_last_45,percent_last_25,trans_amt_mean_0_15,hour_median_2,mcc155,main_count_transaction_amt_12,currency_std_1_mult_percent_last_10,trans_amount_age_interaction_mult_employee_count_nm,main_count_transaction_amt_0_norm,percent_last_10_mult_currency_std_1,percent_last_10_mult_trans_amount_age_interaction,employee_count_nm_mult_report,hour_median_3_mult_main_median_transaction_amt_0,currency_std_1_mult_max_min_diff_dttm,trans_amount_age_interaction,percent_last_90,main_sum_transaction_amt_42,main_median_transaction_amt_0_mult_max_min_diff_dttm,trans_amt_sum_15_30,main_count_transaction_amt_45_norm,transaction_amt__number_crossing_m__m_0,age_count_trans_interaction,main_count_transaction_amt_15_norm,min_day_trans_age_mult_max_min_diff_dttm,hour_median_15,percent_last_60,percent_last_10_mult_max_min_diff_dttm,trans_amt_sum_0_15,main_count_transaction_amt_2_norm,main_median_transaction_amt_0_mult_min_day_trans_age,report_mult_hour_median_3,main_median_transaction_amt_110,"transaction_amt__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0",trans_amount_age_interaction3,employee_count_freq,percent_last_5,min_day_trans_age_mult_percent_last_10,max_min_diff_dttm_mult_main_median_transaction_amt_0,main_sum_transaction_amt_51,currency_daydiff_30_mean_1,employee_count_nm_mult_currency_std_1,percent_last_30,max_min_diff_dttm_mult_trans_amount_age_interaction,transaction_amt__autocorrelation__lag_9,employee_count_nm_mult_main_sum_transaction_amt_12,msumm3,max_min_diff_dttm_mult_main_sum_transaction_amt_12,transaction_amt__fourier_entropy__bins_5,currency_std_1_mult_report,general_trans_info_30_median,mccc51,trans_amt_median_15_30,NUM_UNIQUE(transactions.mcc_code),main_count_transaction_amt_28_norm,percent_last_15,main_count_transaction_amt_9,"transaction_amt__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",mccc3,main_count_transaction_amt_43_norm,hour_count_5_norm,days_between_trans_median,"transaction_amt__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""max""","transaction_amt__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)",hour_count_17_norm,"transaction_amt__fft_coefficient__attr_""imag""__coeff_2",transaction_amt__autocorrelation__lag_5,main_median_transaction_amt_1,transaction_amt__autocorrelation__lag_1,main_count_transaction_amt_26_norm,PERCENTILE(NUM_UNIQUE(transactions.MONTH(transaction_dttm))),main_count_transaction_amt_33_norm,main_sum_transaction_amt_9,percent_last_10_mult_employee_count_nm,count_trans_transaction_amt_sum,"transaction_amt__fft_coefficient__attr_""abs""__coeff_0","transaction_amt__fft_coefficient__attr_""angle""__coeff_16",count_trans_dates,"transaction_amt__fft_coefficient__attr_""real""__coeff_1",employee_count_nm,main_count_transaction_amt_94_norm,main_count_transaction_amt_93_norm,hour_count_1,max_min_diff_dttm_mult_percent_last_10,min_diff_dttm,main_count_transaction_amt_10_norm,transaction_amt__fourier_entropy__bins_2,nunique_mcc_codes,logsumabs_1,min_day_trans_age_mult_main_median_transaction_amt_0,hour_count_23_norm,hour_count_1_norm,general_trans_info_1000_median,num_transaction_before_30_days,main_count_transaction_amt_51_norm,currency_std_1,"transaction_amt__fft_coefficient__attr_""abs""__coeff_51","transaction_amt__fft_coefficient__attr_""real""__coeff_81",transaction_amt__partial_autocorrelation__lag_4,per

In [45]:
# Обучение с добавлением статистических фичей

clients = pd.read_csv('/kaggle/input/ods-churn-24/clients.csv')
clients.replace({'employee_count_nm':{'ОТ 101 ДО 500':1,'БОЛЕЕ 1001':2,'ОТ 501 ДО 1000':3,'ДО 10':4,
                                      'ОТ 11 ДО 50':5,'ОТ 51 ДО 100':6,'БОЛЕЕ 500':7,'ОТ 11 ДО 30':8,
                                      'ОТ 31 ДО 50':9}}, inplace=True)
clients.employee_count_nm = clients.employee_count_nm.fillna(0)
train = pd.read_csv('/kaggle/input/ods-churn-24/train.csv')
clients = clients.merge(train, how='left')
clients.loc[clients.target.isna(), 'target'] = -1

cur = clients[clients.target != -1]
cur = cur.groupby(['customer_age', 'employee_count_nm'])['target'].agg(['mean']).reset_index().rename({'mean':'group_employee_age_mean'}, axis=1)
clients = clients.merge(cur, how='left', on=['customer_age', 'employee_count_nm'])
clients.loc[clients.target != -1, 'group_employee_age_mean'] = -1

cur = clients[clients.target != -1]
cur = cur.groupby(['report', 'customer_age'])['target'].agg(['mean']).reset_index().rename({'mean':'group_report_age_mean'}, axis=1)
clients = clients.merge(cur, how='left', on=['report', 'customer_age'])
clients.loc[clients.target != -1, 'group_report_age_mean'] = -1

cur = clients[clients.target != -1]
cur = cur.groupby(['report', 'employee_count_nm'])['target'].agg(['mean']).reset_index().rename({'mean':'group_report_employee_mean'}, axis=1)
clients = clients.merge(cur, how='left', on=['report', 'employee_count_nm'])
clients.loc[clients.target != -1, 'group_report_employee_mean'] = -1

df = df.merge(clients[['user_id', 'group_employee_age_mean', 'group_report_age_mean', 'group_report_employee_mean', 'report', 'customer_age']], how='left')
cat_cols += ['group_employee_age_mean', 'group_report_age_mean', 'group_report_employee_mean']
good_cols += ['group_employee_age_mean', 'group_report_age_mean', 'group_report_employee_mean']
df[cat_cols] = df[cat_cols].astype(str)
train = df[df.target != -1]
test = df[df.target == -1]

In [49]:
# Обучение основных моделей на 5 Фолдах.

strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

dr_cols = ['group_employee_age_mean', 'group_report_age_mean', 'group_report_employee_mean']

X, y = train.drop(['time']+dr_cols, axis=1), train['target']
scores = []
models = []
for train_index, valid_index in strat_kfold.split(train, train['target']):
    
    X_train, X_val = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_val, target_val = y.iloc[train_index], y.iloc[valid_index], train['target'].iloc[valid_index]

    # Для того чтобы не было лика, генерируем статистические фичи только на данных для тренировки
    cur = X_train[['employee_count_nm', 'customer_age', 'target']].groupby(['customer_age', 'employee_count_nm'])['target'].agg(['mean']).reset_index().rename({'mean':'group_employee_age_mean'}, axis=1)
    X_train = X_train.merge(cur, how='left', on=['customer_age', 'employee_count_nm'])
    X_val = X_val.merge(cur, how='left', on=['customer_age', 'employee_count_nm'])
    
    cur = X_train[['report', 'customer_age', 'target']].groupby(['report', 'customer_age'])['target'].agg(['mean']).reset_index().rename({'mean':'group_report_age_mean'}, axis=1)
    X_train = X_train.merge(cur, how='left', on=['report', 'customer_age'])
    X_val = X_val.merge(cur, how='left', on=['report', 'customer_age'])
    
    cur = X_train[['report', 'employee_count_nm', 'target']].groupby(['report', 'employee_count_nm'])['target'].agg(['mean']).reset_index().rename({'mean':'group_report_employee_mean'}, axis=1)
    X_train = X_train.merge(cur, how='left', on=['report', 'employee_count_nm'])
    X_val = X_val.merge(cur, how='left', on=['report', 'employee_count_nm'])
    
    X_train = X_train.drop(['report', 'customer_age'], axis=1)
    X_val = X_val.drop(['report', 'customer_age'], axis=1)
    X_train[cat_cols] = X_train[cat_cols].astype(str)
    X_val[cat_cols] = X_val[cat_cols].astype(str)
    
    
    model = CatBoostClassifier(
        iterations = 2500,
        depth=4,
        learning_rate=0.03,
        eval_metric='AUC',
        cat_features = cat_cols,
        early_stopping_rounds=400,

    )


    model.fit(Pool(X_train[good_cols], y_train, cat_features=cat_cols),
              eval_set=Pool(X_val[good_cols], y_val, cat_features=cat_cols),
              verbose=400)
    models.append(model)
    
    pred = model.predict_proba(X_val[good_cols])[:, 1]
    scores.append(metrics.roc_auc_score(y_val, pred))

print('\n', np.mean(scores))

0:	test: 0.6926407	best: 0.6926407 (0)	total: 281ms	remaining: 16m 21s
400:	test: 0.7516215	best: 0.7516215 (400)	total: 1m 31s	remaining: 11m 47s
800:	test: 0.7599511	best: 0.7599511 (800)	total: 3m 3s	remaining: 10m 18s
1200:	test: 0.7639699	best: 0.7639699 (1200)	total: 4m 36s	remaining: 8m 48s
1600:	test: 0.7664714	best: 0.7664714 (1600)	total: 6m 8s	remaining: 7m 16s
2000:	test: 0.7681964	best: 0.7682050 (1997)	total: 7m 38s	remaining: 5m 43s
2400:	test: 0.7696749	best: 0.7696749 (2400)	total: 9m 10s	remaining: 4m 12s
2800:	test: 0.7708321	best: 0.7708321 (2800)	total: 10m 41s	remaining: 2m 40s
3200:	test: 0.7717648	best: 0.7717648 (3200)	total: 12m 10s	remaining: 1m 8s
3499:	test: 0.7723044	best: 0.7723044 (3499)	total: 13m 15s	remaining: 0us

bestTest = 0.7723043859
bestIteration = 3499

0:	test: 0.6924720	best: 0.6924720 (0)	total: 260ms	remaining: 15m 8s
400:	test: 0.7603672	best: 0.7603672 (400)	total: 1m 30s	remaining: 11m 42s
800:	test: 0.7688256	best: 0.7688256 (800)	total

In [ ]:
# 0.7827116993586097 -> 0.7858941983144541 на всех наших фичах
# 0.7837420539594062 -> 0,7858268987 с добавлением фич от генератора Марка на модиф-х посл-х
# 0.7827997926298924 -> 0,7861547167 с добавлением фич из моего базовго ноута, но с модификациями транзакций
# 0.7828451780588307 -> 0,7859032167 если еще добавить фичи из 6.7 и 6.8

In [47]:
# На основе всех фолдов выбираем самые важные признаки

imp = pd.DataFrame({'name': X_train[good_cols].columns.to_list()})

for i in range(len(models)):
    imp_i = pd.DataFrame({
        'name': X_train[good_cols].columns.to_list(),
        str(i): models[i].get_feature_importance()
    })
    imp = imp.merge(imp_i, how='left', on='name')

imp['total'] = imp[[str(i) for i in range(len(models))]].sum(axis=1)
imp = imp.sort_values('total', ascending=False)
top_features = list(imp[imp.total>20]['name'].values)
imp[0:30]


,name,0,1,2,3,4,total
389,mega_features,2.184999,1.561162,1.864357,1.651995,2.709883,9.972395
162,main_65_84,1.673018,1.527992,1.169216,1.055256,1.081267,6.506748
404,^2_avg_trans_per_mcc_mult_^5_currency_max_1,0.780457,0.934019,0.699619,1.042228,0.860034,4.316358
193,seq_3_month81_mean,0.929906,0.758577,0.949147,0.898375,0.686999,4.223003
151,main_62_1,0.875304,0.639252,0.939459,0.928891,0.836412,4.219316
2,"CUM_SUM(TREND(transactions.transaction_amt, tr...",0.601054,0.801618,0.845752,1.118437,0.647745,4.014607
71,main_count_transaction_amt_28_norm,1.006159,0.613015,0.611293,0.795673,0.833044,3.859185
16,trans_amt_sum_0_15_ratio,0.589744,0.903043,0.793867,0.837954,0.617130,3.741738
442,group_report_employee_mean,0.729690,0.867018,0.630799,0.932550,0.522239,3.682295
174,main_68_160,0.654909,0.624509,0.604255,0.840964,0.911085,3.635722


In [48]:
sample = pd.read_csv('/kaggle/input/ods-churn-24/sample_submit_naive.csv')
sample['predict'] = 0
for i in range(len(models)):
    sample['predict'] += models[i].predict_proba(test[good_cols])[:, 1]

sample.to_csv('submission_ctb_features_union_add_seq_m_f_seq_base_f.csv', index=False)
sample

,user_id,predict
0,9,0.117344
1,61,0.057685
2,62,0.085547
3,80,0.027794
4,88,0.507501
...,...,...
31995,561362,0.369080
31996,561419,0.366794
31997,561895,0.337381
31998,561908,0.657650
